In [2]:
! pip install tweepy
! pip install numpy
! pip install pandas

     |████████████████████████████████| 147 kB 1.4 MB/s eta 0:00:01
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 15.3 MB 1.2 MB/s eta 0:00:01
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 9.9 MB 1.7 MB/s eta 0:00:01     |██████████████▉                 | 4.6 MB 1.5 MB/s eta 0:00:04
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [3]:
import os
import tweepy
import numpy as np
import pandas as pd

In [5]:
TWITTER_API_KEY = os.environ.get('TWITTER_API_KEY')
TWITTER_API_SECRET = os.environ.get('TWITTER_API_SECRET')

In [6]:
auth = tweepy.OAuthHandler(TWITTER_API_KEY, TWITTER_API_SECRET)

In [7]:
api = tweepy.API(auth)

In [8]:
tp = api.get_user('franciscojarceo')

In [9]:
tp._json

{'id': 1145326260341682176,
 'id_str': '1145326260341682176',
 'name': 'Francisco Javier Arceo',
 'screen_name': 'franciscojarceo',
 'location': 'USA',
 'profile_location': None,
 'description': 'Building the @fast-est data science on the internet. \n\nPreviously @GoldmanSachs @CommBank @AIG. Alum @Columbia Engineering.',
 'url': 'https://t.co/jCS8iVUQAA',
 'entities': {'url': {'urls': [{'url': 'https://t.co/jCS8iVUQAA',
     'expanded_url': 'https://franciscojavierarceo.github.io/',
     'display_url': 'franciscojavierarceo.github.io',
     'indices': [0, 23]}]},
  'description': {'urls': []}},
 'protected': False,
 'followers_count': 1189,
 'friends_count': 3159,
 'listed_count': 17,
 'created_at': 'Sun Jun 30 13:40:34 +0000 2019',
 'favourites_count': 24152,
 'utc_offset': None,
 'time_zone': None,
 'geo_enabled': True,
 'verified': False,
 'statuses_count': 8143,
 'lang': None,
 'status': {'created_at': 'Fri Mar 26 11:19:57 +0000 2021',
  'id': 1375407167998734337,
  'id_str': '137

In [10]:
def get_all_tweets(screen_name):
    #initialize a list to hold all the tweepy Tweets
    alltweets = []  

    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name, count=200)

    #save most recent tweets
    alltweets.extend(new_tweets)

    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1

    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print(f"getting tweets before {oldest}")
        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200, max_id=oldest)
        #save most recent tweets
        alltweets.extend(new_tweets)
        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
        print(f"...{len(alltweets)} tweets downloaded so far")

    return alltweets

In [11]:
mytweets = get_all_tweets('franciscojarceo')

getting tweets before 1367559747474501636
...395 tweets downloaded so far
getting tweets before 1364255309825413119
...593 tweets downloaded so far
getting tweets before 1360359218071695364
...793 tweets downloaded so far
getting tweets before 1358166987424292864
...989 tweets downloaded so far
getting tweets before 1355135491046313984
...1189 tweets downloaded so far
getting tweets before 1353728150837014531
...1387 tweets downloaded so far
getting tweets before 1352016044597125119
...1586 tweets downloaded so far
getting tweets before 1349845689065418752
...1786 tweets downloaded so far
getting tweets before 1348676065875656704
...1983 tweets downloaded so far
getting tweets before 1346868068388122626
...2183 tweets downloaded so far
getting tweets before 1343972297644773375
...2383 tweets downloaded so far
getting tweets before 1341736022170726399
...2583 tweets downloaded so far
getting tweets before 1339965273357443071
...2783 tweets downloaded so far
getting tweets before 1337258

In [12]:
mytweets[2]._json['in_reply_to_status_id']

1375276847668285440

In [13]:
mytweets[1]._json['in_reply_to_status_id']

In [14]:
mytweets[1]._json['id'] #['text']

1375294255443320838

In [15]:
'in_reply_to_status_id' in mytweets[1]._json

True

In [16]:
res = []
for i in mytweets:
    if i._json['in_reply_to_status_id'] is not None:
        res.append((i._json['id'], i._json['text']))

In [17]:
res[0:4]

[(1375277503007318020, 'i ate 2 sleeves 😂😂😂😭😭😭'),
 (1375275248141090816,
  '@zevdatascience @ecologyofgavin @DhruvChandAgga1 lmaooooo'),
 (1375201505863725061, '@iamjakestream cc @brunoswerneck'),
 (1375195927380697093, '@MAustin4249 https://t.co/NVo5wSAvu6')]